In [1]:
import pandas as pd
import numpy as np
import sentence_transformers

## Creating sentence embeddings

In [2]:
 model = sentence_transformers.SentenceTransformer("../inputData/sentence_transformer_nli/model_results")

In [3]:
model.cuda()

SentenceTransformer(
  (0): BERT(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [4]:
import os
PATH = '/home/axhue/Documents/Kaggle_Competitions/common/inputData/v6_text/'
def str_num_sort(inp_str):
    return int(inp_str.split('_')[-1].split('.')[0])

df_pickles = [PATH + p for p in os.listdir(PATH)]
df_pickles.sort(key=str_num_sort)

mini_df = pd.read_pickle(df_pickles[0],compression="gzip")

file_ids = mini_df['paper_id'].unique()
abs_df = mini_df[mini_df['section'] == 'abstract']

In [5]:
res_embeds = model.encode(abs_df['sentence'].tolist())

In [6]:
res_np = np.array(res_embeds)

In [7]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from plotly import express
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [21]:
''.join(abs_df[abs_df['paper_id'] == file_ids[2]]['sentence'].values)

'Progressive multifocal leukoencephalopathy (PML) is a severely debilitating and often fatal demyelinating disease of the central nervous system (CNS) in immunosuppressed individuals caused by JC polyomavirus (JCV), a ubiquitous human pathogen.Demyelination results from lytically infected oligodendrocytes, whose clearance is impaired in the setting of depressed JCV-specific T cell-mediated CNS surveillance.Although mutations in the viral capsid and genomic rearrangements in the viral non-coding region appear to set the stage for PML in the immunosuppressed population, mechanisms of demyelination and CNS antiviral immunity are poorly understood in large part due to absence of a tractable animal model that mimics PML neuropathology in humans.Early studies using mouse polyomavirus (MPyV) in T cell-deficient mice demonstrated productive viral replication in the CNS and demyelination; however, these findings were confounded by spinal cord compression by virus-induced vertebral bone tumors.H

In [43]:
a = mini_df[mini_df['paper_id'] == file_ids[9]]

a.iloc[0]['sentence']

'Type 1 Interferons and NK Cells Limit Murine Cytomegalovirus Escape from the Lymph Node Subcapsular Sinus'

In [ ]:
''.join(abs_df[abs_df['paper_id'] == file_ids[2]]

In [9]:
scale_res = scaler.fit_transform(res_np)
pca = PCA(n_components = 0.92)
pca_res = pca.fit_transform(scale_res)

In [29]:
tsne_res = TSNE(n_components=2).fit_transform(pca_res)

In [30]:
tsne_df = pd.DataFrame(tsne_res,columns = ['x','y'])
tsne_df = tsne_df.assign(paper_id=abs_df['paper_id'].reset_index(drop=True),sentence=abs_df['sentence'].reset_index(drop=True))

In [41]:
tsne_df.iloc[0,3]

'Background: Entomological surveys of Simulium vectors are an important component in the criteria used to determine if Onchocerca volvulus transmission has been interrupted and if focal elimination of the parasite has been achieved.'

In [42]:
fig = express.scatter(tsne_df[tsne_df['paper_id'].isin(file_ids[:2])],x='x',y='y',color='paper_id',hover_data=['paper_id'])
fig.show()

ValueError: PandasArray must be 1-dimensional.

# Clustering

## Kmeans (not great)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
#K means Clustering 
def doKmeans(X, nclust=2):
    model = KMeans(nclust)
    model.fit(X)
    clust_labels = model.predict(X)
    cent = model.cluster_centers_
    return (clust_labels, cent)

clust_labels, cent = doKmeans(fres1, 10)
kmeans = pd.DataFrame(clust_labels)

In [ ]:
tsne_df = tsne_df.assign(cluster=kmeans)

In [ ]:
fig = express.scatter(tsne_df,x='x',y='y',color='cluster')
fig.show()

In [ ]:
tsne_df = tsne_df.assign(sentence=abs_df['sentence'].reset_index(drop=True))

Using tSNE yield no distinctive groups, we can attempt using different clustering methods but the only other option for clustering would be to cluster

In [67]:
tsne_df.iloc[9343]

x                                            60.9979
y                                           -46.0571
paper_id    ce40c2d0956a6167604ab1a0955fd47f22b55111
cluster                                            3
sentence                                          []
Name: 9343, dtype: object

## DBSCAN

In [23]:
scale_res = scaler.fit_transform(res_np)
pca = PCA(n_components = 0.94)
pca_res = pca.fit_transform(scale_res)

In [24]:
pca_res.shape

(9366, 99)

In [25]:
from sklearn.cluster import DBSCAN

In [48]:
db = DBSCAN(eps=0.1, min_samples=7).fit(pca_res)
set(db.labels_)

{-1, 0, 1, 2, 3, 4, 5}

In [49]:
tsne_res = TSNE(n_components=2).fit_transform(pca_res)

In [50]:
tsne_df = pd.DataFrame(tsne_res,columns = ['x','y'])
tsne_df = tsne_df.assign(paper_id=abs_df['paper_id'].reset_index(drop=True),cluster=db.labels_)

In [51]:
fig = express.scatter(tsne_df,x='x',y='y',color='cluster')
fig.show()